<a href="https://colab.research.google.com/github/lalroshan590/.github/blob/master/adk_deployment_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Agent Development: From Deployment to Evaluation on Vertex AI

Welcome, developers! 🚀

This notebook is your comprehensive guide to the full lifecycle of building, deploying, and evaluating sophisticated AI agents using the Google Agent Development Kit (ADK) and Vertex AI.

**Our journey will follow a specific, educational flow:**
1.  **Agent #1 (Standard Deployment)**: We'll create a multi-agent trip planner with **custom Python tools** and deploy it *without* tracing to see a baseline deployment.
2.  **Understanding Sessions**: We'll query our first agent with and without sessions to clearly demonstrate the difference between stateless and stateful (short-term memory) conversations.
3.  **Agent #2 (Memory & Tracing)**: We'll build a second agent with a long-term **Memory Bank** and deploy it *with* **tracing** enabled.
4.  **Comparing Tracing**: We'll see the practical difference in observability between an agent deployed with and without tracing.
5.  **Comprehensive Evaluation**: Finally, we'll run a rigorous evaluation on our first agent to measure its performance, tool selection, and even the accuracy of the arguments passed to its tools.

Let's get started!

## Author

HI, I'm Qingyue (Annie) Wang, a developer advocate and AI engineer at **Google**, passionate about helping developers build with AI and cloud technologies :)


If you have questions with this notebook, contact me on [LinkedIn](https://www.linkedin.com/in/qingyuewang/) , [X](https://twitter.com/qingyuewang) or email anniewangtech0510@Gmail.com


```
  (\__/)
  (•ㅅ•)
  /づ  📚      Enjoy learning AI Agents :)
```


-------------
### 🎁 🛑 Important Prerequisite: Setup Your Environment! 🛑 🎁
-----------------------------------------------------------------------------

👉 **Set Up Your GCP HERE**: https://codelabs.developers.google.com/onramp/instructions#1

 -----------------------------------------------------------------------------

---

## 1. Setup and Configuration

First, we'll install the necessary packages and configure our Google Cloud environment.

### Install Libraries
This command installs the Google Cloud AI Platform SDK, including support for the Agent Development Kit (`adk`), evaluation, and Agent Engines.

In [ ]:
# --- 1. Installation ---
%pip install --upgrade --quiet 'google-cloud-aiplatform[evaluation,adk,agent_engines]'
%pip install --upgrade --quiet 'google-api-python-client'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.5/229.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.8/103.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.6/118.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.3/239.3 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21

### Import Libraries
Now, let's import all the libraries we'll need for this session.

In [ ]:
# --- 2. Import all necessary libraries ---
import os
import sys
import json
import asyncio
import random
import string
from uuid import uuid4
from typing import Any, List

import pandas as pd
import plotly.graph_objects as go
import vertexai
from google.colab import auth
from IPython.display import HTML, Markdown, display

# --- ADK, Agent, and Evaluation Components ---
from google.adk.agents import Agent
from google.adk.events import Event
from google.adk.runners import Runner
import google.adk as adk
from google.adk.memory import VertexAiMemoryBankService
from google.adk.sessions import VertexAiSessionService, InMemorySessionService
from google.genai import types
from vertexai import agent_engines
from vertexai.preview import reasoning_engines
from vertexai.preview.evaluation import EvalTask

print("✅ All libraries are ready to go!")

✅ All libraries are ready to go!


### Authenticate and Configure Your Project
To use Vertex AI, you need an active Google Cloud project. This section handles authenticating your environment and setting up the necessary project configurations.

In [ ]:
# --- 3. Authentication & Project Configuration ---

# Authenticate user in Colab
if "google.colab" in sys.modules:
    auth.authenticate_user()
    print("✅ Authenticated successfully.")

# @title Set Your Google Cloud Project Details
PROJECT_ID = "your-project-id"  # @param {type:"string"}
LOCATION = "us-central1"          # @param {type:"string"}
STAGING_BUCKET = f"gs://{PROJECT_ID}-adk-staging" # A unique name for your bucket

# Set environment variables for the ADK and gcloud
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "True"

# Create the staging bucket if it doesn't exist
!gcloud storage buckets create {STAGING_BUCKET} --project={PROJECT_ID} --location={LOCATION} -l {LOCATION} 2>/dev/null

# Initialize the Vertex AI SDK
# We'll also create an "Experiment" to group our evaluation runs
EVAL_EXPERIMENT_NAME = f"trip-planner-evaluation-{uuid4().hex[:6]}"
vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET, experiment=EVAL_EXPERIMENT_NAME)

print(f"\n✅ Vertex AI configured for project '{PROJECT_ID}' in '{LOCATION}'.")
print(f"🔬 Evaluation results will be saved to experiment: {EVAL_EXPERIMENT_NAME}")

✅ Authenticated successfully.



✅ Vertex AI configured for project 'neon-emitter-458622-e3' in 'us-central1'.
🔬 Evaluation results will be saved to experiment: trip-planner-evaluation-7f618d


---

## 2. Agent #1: The Trip Planner (Standard Deployment)

We'll build our first agent, a "team" of specialists. Crucially, we will now provide them with our own **custom Python functions as tools** instead of a generic search tool. This makes their behavior deterministic and much easier to evaluate accurately.

### Define Custom Tools

These functions act as our mock APIs or databases. The agent will learn to call these with the correct parameters based on the user's request.

In [ ]:
# --- Custom Tool Definitions ---

def get_travel_attractions(location: str, interest: str = "any"):
    """
    Finds travel attractions in a given location based on an interest.

    Args:
        location: The city or place to search for attractions (e.g., "Paris", "Rome").
        interest: The type of attraction to look for (e.g., "museum", "historical site", "park").
    """
    # This is a mock database for demonstration
    attractions = {
        "paris": {"museum": "The Louvre", "historical site": "Eiffel Tower"},
        "rome": {"historical site": "The Colosseum", "museum": "Vatican Museums"},
        "tokyo": {"museum": "teamLab Borderless", "park": "Shinjuku Gyoen"},
        "san francisco": {"historical site": "Alcatraz Island", "park": "Golden Gate Park"}
    }

    city = attractions.get(location.lower(), {})
    if interest == "any":
        return city or f"Sorry, I don't have any information for {location}."
    else:
        return city.get(interest.lower(), f"Sorry, I couldn't find a {interest} in {location}.")

def get_restaurant_recommendations(location: str, cuisine: str):
    """
    Recommends a restaurant in a location based on the desired cuisine.

    Args:
        location: The city where the user wants to eat (e.g., "Paris", "Rome").
        cuisine: The type of food they want (e.g., "pasta", "seafood", "cafe").
    """
    # This is a mock database for demonstration
    restaurants = {
        "rome": {"pasta": "Trattoria Da Enzo al 29"},
        "san francisco": {"seafood": "Hog Island Oyster Co."},
        "paris": {"cafe": "Les Deux Magots", "fancy": "Le Cinq"}
    }

    city = restaurants.get(location.lower(), {})
    return city.get(cuisine.lower(), f"Sorry, I couldn't find a {cuisine} restaurant in {location}.")

### Define and Deploy the Agent Team (Without Tracing)
Now, we'll update our specialist agents to use these new, more specific tools.

In [ ]:
# --- Specialist Agent Definitions using Custom Tools ---
def create_activities_agent():
    return Agent(
        name="activities_agent", model="gemini-2.5-flash",
        description="Specialist for finding interesting activities, attractions, and historical sites.",
        tools=[get_travel_attractions],
    )

def create_food_agent():
    return Agent(
        name="food_agent", model="gemini-2.5-flash",
        description="Specialist for finding great restaurants based on cuisine.",
        tools=[get_restaurant_recommendations],
    )

# --- Coordinator Agent Definition ---
def create_trip_coordinator_agent(sub_agents: List[Agent]):
    return Agent(
        name="trip_coordinator_agent", model="gemini-2.5-pro",
        description="The master coordinator for trip planning. It delegates tasks to specialist agents.",
        sub_agents=sub_agents,
        instruction="""
        You are the master "Trip Coordinator" 🧭. Your job is to plan a full itinerary by delegating to your specialist agents.
        Extract the location, interests, and cuisine types from the user's prompt to use as arguments for your specialist agents.
        Once you have the information, combine it into a single, cohesive itinerary and present it in a clean MARKDOWN format.
        """,
    )


In [ ]:
# --- Assemble and Deploy the Team ---
trip_coordinator_agent = create_trip_coordinator_agent(
    sub_agents=[create_activities_agent(), create_food_agent()]
)
print(f"👑 Agent '{trip_coordinator_agent.name}' is ready for deployment!")

deployment_name = f"trip-planner-basic-{uuid4().hex[:6]}"
print(f"🚀 Deploying agent as '{deployment_name}' without tracing...")

deployed_agent_no_trace = agent_engines.create(
    display_name=deployment_name,
    agent_engine=trip_coordinator_agent, # Pass the agent object directly
    requirements=["google-cloud-aiplatform[adk,agent_engines]"],
    description="A multi-agent trip planning system with custom tools and tracing disabled."
)

print("-" * 50)
print("✅ Agent System deployed successfully!")
display(Markdown(f"**Resource Name:** `{deployed_agent_no_trace.resource_name}`"))

INFO:vertexai.agent_engines:Deploying google.adk.agents.Agent as an application.
INFO:vertexai.agent_engines:Identified the following requirements: {'google-cloud-aiplatform': '1.105.0', 'pydantic': '2.11.7', 'cloudpickle': '3.1.1'}
INFO:vertexai.agent_engines:The following requirements are appended: {'pydantic==2.11.7', 'cloudpickle==3.1.1'}
INFO:vertexai.agent_engines:The final list of requirements: ['google-cloud-aiplatform[adk,agent_engines]', 'pydantic==2.11.7', 'cloudpickle==3.1.1']


👑 Agent 'trip_coordinator_agent' is ready for deployment!
🚀 Deploying agent as 'trip-planner-basic-fa6b23' without tracing...


INFO:vertexai.agent_engines:Using bucket neon-emitter-458622-e3-adk-staging
INFO:vertexai.agent_engines:Wrote to gs://neon-emitter-458622-e3-adk-staging/agent_engine/agent_engine.pkl
INFO:vertexai.agent_engines:Writing to gs://neon-emitter-458622-e3-adk-staging/agent_engine/requirements.txt
INFO:vertexai.agent_engines:Creating in-memory tarfile of extra_packages
INFO:vertexai.agent_engines:Writing to gs://neon-emitter-458622-e3-adk-staging/agent_engine/dependencies.tar.gz
INFO:vertexai.agent_engines:Creating AgentEngine
INFO:vertexai.agent_engines:Create AgentEngine backing LRO: projects/680476413759/locations/us-central1/reasoningEngines/4966159771129348096/operations/3349422995880804352
INFO:vertexai.agent_engines:View progress and logs at https://console.cloud.google.com/logs/query?project=neon-emitter-458622-e3
INFO:vertexai.agent_engines:AgentEngine created. Resource name: projects/680476413759/locations/us-central1/reasoningEngines/4966159771129348096
INFO:vertexai.agent_engines:

--------------------------------------------------
✅ Agent System deployed successfully!


**Resource Name:** `projects/680476413759/locations/us-central1/reasoningEngines/4966159771129348096`

---

## 3. Interacting with Agent #1: Understanding Sessions

Now we'll query our deployed agent. The goal here is to understand the crucial difference between a single, stateless query and a stateful conversation that uses a session for short-term memory.

### Helper Function for Clean Interaction
To avoid repeating code, we'll create one reusable helper function.

In [ ]:
def stream_and_print_response(stream_generator) -> str:
    """
    Streams a response from an agent, prints it in real-time, and returns the full text.

    Args:
        stream_generator: The generator object returned by an agent's stream_query().

    Returns:
        The complete, concatenated response string.
    """
    full_response = []
    print("🤖 Agent Response: ", end="")
    # Iterate through all events in the stream
    for chunk in stream_generator:
        # The key is to check for content from the 'model' role.
        # This filters out intermediate tool calls and other events.
        if chunk.get('content') and chunk['content'].get('role') == 'model':
            try:
                # Safely access the nested text content
                text_content = chunk['content']['parts'][0]['text']
                if text_content:
                    print(text_content, end="", flush=True)
                    full_response.append(text_content)
            except (KeyError, IndexError):
                # Ignore model chunks that might not have text (rare)
                pass

    print() # Adds a newline for clean formatting
    return "".join(full_response)

### Querying Without a Session (Stateless)
This is a single-shot question. The agent has no memory before or after this query. **The code is now corrected to include the required `user_id` to fix the `FailedPrecondition` error.**

In [ ]:
print("🗣️ Querying WITHOUT a session (Stateless)")
query1 = "My name is Annie. Plan a day in San Francisco. I want to visit a famous historical site in the morning, and then find a great place for seafood for dinner."
print(f"\n💬 User: {query1}")

# even for stateless queries.
user_id_stateless = f"stateless-user-{uuid4().hex[:6]}"
stream1 = deployed_agent_no_trace.stream_query(
    message=query1,
    user_id=user_id_stateless
)
full_response1 = stream_and_print_response(stream1)

print("\n" + "-" * 50)

query2 = "What is my name?"
print(f"\n💬 User: {query2}")

stream2 = deployed_agent_no_trace.stream_query(
    message=query2,
    user_id=user_id_stateless
)
full_response2 = stream_and_print_response(stream2)

print("\n" + "-" * 50)


🗣️ Querying WITHOUT a session (Stateless)

💬 User: My name is Annie. Plan a day in San Francisco. I want to visit a famous historical site in the morning, and then find a great place for seafood for dinner.
🤖 Agent Response: Annie, I recommend Hog Island Oyster Co. for great seafood in San Francisco.

Of course, Annie! Here is a sample itinerary for your day in San Francisco:

### **Your San Francisco Itinerary**

***

#### **Morning: Historical Exploration**

*   **Visit Alcatraz Island:** I'd recommend starting your day with a trip to one of San Francisco's most famous historical sites. The ferry ride offers beautiful views of the bay and the Golden Gate Bridge. Be sure to book your tickets in advance as they sell out quickly!

#### **Afternoon: Waterfront Stroll**

*   **Explore Fisherman's Wharf:** After your tour of Alcatraz, you'll be right at Fisherman's Wharf. You can explore the various shops, see the sea lions at Pier 39, and grab a light snack.

#### **Evening: Seafood Dinne

### Querying With a Session (Stateful)
Now, we'll use a session to enable short-term memory for a single, continuous conversation.

In [ ]:
print("\n\n🗣️ Querying WITH a session (Stateful Conversation)")

user_id = f"session-demo-user-{uuid4().hex[:6]}"
session = deployed_agent_no_trace.create_session(user_id=user_id)
print(f"✅ Reusable session created: {session['id']}")

# --- First Query ---
query_1 = "My name is Annie. I want to plan a trip to Rome."
print(f"\n💬 User: {query_1}")
stream_1 = deployed_agent_no_trace.stream_query(message=query_1, user_id=user_id, session_id=session['id'])
response_1 = stream_and_print_response(stream_1)

# --- Follow-up Query (in the same session) ---
query_2 = "Find a famous historical site there and a good place for pasta."
print(f"\n💬 User: {query_2}")
stream_2 = deployed_agent_no_trace.stream_query(message=query_2, user_id=user_id, session_id=session['id'])
response_2 = stream_and_print_response(stream_2)

# --- Follow-up Query (in the same session) ---
query_3 = "What is my name?"
print(f"\n💬 User: {query_3}")
stream_3 = deployed_agent_no_trace.stream_query(message=query_3, user_id=user_id, session_id=session['id'])
response_3 = stream_and_print_response(stream_3)

# --- Follow-up Query (in the different session) ---
session1 = deployed_agent_no_trace.create_session(user_id=user_id)
print(f"\n💬 User: {query_3} Now in different session")
stream_4 = deployed_agent_no_trace.stream_query(message=query_3, user_id=user_id, session_id=session1['id'])
response_4 = stream_and_print_response(stream_4)

print("\n" + "-" * 50)
print("Success! The agent remembered the location ('Rome') from the first message to correctly answer the second.")



🗣️ Querying WITH a session (Stateful Conversation)
✅ Reusable session created: 8742553952760889344

💬 User: My name is Annie. I want to plan a trip to Rome.
🤖 Agent Response: Excellent! I can certainly help you plan your trip to Rome. To make it the best trip ever, could you tell me a bit more about your interests? For example, are you interested in history, art, food, or something else? Also, what types of cuisine do you enjoy?

💬 User: Find a famous historical site there and a good place for pasta.
🤖 Agent Response: Here is a suggested itinerary for your trip to Rome, Annie:

### Historical Site

A must-see historical site in Rome is the **Colosseum**. This ancient amphitheater is a symbol of Imperial Rome and is one of the most famous landmarks in the world. You can almost hear the roar of the crowds and the clash of gladiators as you walk through its massive arches.

### Authentic Pasta

For a fantastic pasta experience, I recommend **Trattoria Da Enzo al 29**. This charming rest

---

## 4. Agent #2: Adding Long-Term Memory & Tracing
Session memory is temporary. For an agent to remember preferences across *different conversations*, it needs a **Memory Bank**.

Let's build a new agent designed for this. This time, we **will enable tracing** so we can see its inner workings.

### Define and Deploy the Memory Agent (With Tracing)

In [ ]:
from google.genai.types import Content, Part

def create_memory_trip_agent():
    """Creates a trip planner that uses a long-term memory bank."""
    return Agent(
        name="memory_trip_agent", model="gemini-2.5-flash",
        instruction="""You are a helpful trip planning assistant with a perfect memory.
        Your Core Instructions:
        1. At the start of every conversation, use your memory to recall any user preferences.
        2. When the user tells you a new preference (e.g., "I'm a vegetarian"), save this fact to your memory.
        3. When planning, always incorporate the user's remembered preferences.
        """,
        tools=[adk.tools.preload_memory_tool.PreloadMemoryTool()],
    )

memory_agent = create_memory_trip_agent()
print("🧠 Agent with long-term memory is defined.")

🧠 Agent with long-term memory is defined.


In [ ]:
# --- 9. Configure and Run the Local Agent with Memory Bank ---
async def run_memory_test():
    # We will attach our new memory bank to the Agent Engine we already deployed.
    agent_engine_id = deployed_agent_no_trace.name
    app_name = memory_agent.name # Use the agent's name as the app identifier

    print(f"✅ Attaching Memory & Session services to Agent Engine: {agent_engine_id}")

    # Configure services to point to our existing deployment
    memory_bank_service = VertexAiMemoryBankService(
        project=PROJECT_ID, location=LOCATION, agent_engine_id=agent_engine_id
    )
    session_service = VertexAiSessionService(
        project=PROJECT_ID, location=LOCATION, agent_engine_id=agent_engine_id
    )

    # Initialize the local runner once
    runner = Runner(
        agent=memory_agent,
        app_name=app_name,
        session_service=session_service,
        memory_service=memory_bank_service,
    )

    # ---
    # Helper function to run a single turn in a chat
    async def run_turn(query: str, user_id: str, session_id: str):
        print(f"\n🗣️ You: {query}")
        print("🤖 Assistant: ", end="", flush=True) # Print prefix for streaming

        final_response_parts = []
        async for event in runner.run_async(
            user_id=user_id,
            session_id=session_id,
            new_message=Content(parts=[Part(text=query)], role="user")
        ):
            # ⭐️ FIX: Access event data using dot notation for objects.
            if event.content and event.content.role == "model":
                part_text = event.content.parts[0].text
                if part_text:
                    final_response_parts.append(part_text)
                    print(part_text, end="", flush=True) # Print streaming parts on one line

        print() # Add a final newline after the response is complete
        return "".join(final_response_parts)
    # ---

    # == Step 1: The "Teach Me" Conversation ==
    user_id_memory = f"adk-traveler-{uuid4()}"
    print(f"\n--- Starting Session 1 for user '{user_id_memory}' ---")
    session_one = await session_service.create_session(user_id=user_id_memory, app_name=app_name)
    await run_turn("Hi there. When you help me plan trips, please remember that I am a vegetarian.", user_id_memory, session_one.id)

    # ⭐️ IMPORTANT: Explicitly save the completed session to long-term memory
    print("\n💾 Saving session 1 to Memory Bank...")
    completed_session = await session_service.get_session(app_name=app_name, user_id=user_id_memory, session_id=session_one.id)
    await memory_bank_service.add_session_to_memory(completed_session)
    print("✅ Session 1 saved.")

    # == Step 2: The "Recall" Conversation ==
    print(f"\n--- Starting a brand new Session 2 for the SAME user ---")
    session_two = await session_service.create_session(user_id=user_id_memory, app_name=app_name)
    await run_turn("Great. Now, can you plan a weekend dinner trip to Napa Valley?", user_id_memory, session_two.id)
    print("-" * 50)
    print("🎉 Memory test complete!")

# In a notebook cell, run the test with await
await run_memory_test()

✅ Attaching Memory & Session services to Agent Engine: 4966159771129348096

--- Starting Session 1 for user 'adk-traveler-65c6a450-92d2-4e3d-99e3-9df8e6cc51f8' ---

🗣️ You: Hi there. When you help me plan trips, please remember that I am a vegetarian.
🤖 Assistant: Hi there! Thanks for letting me know. I've saved that you are a vegetarian to my memory. I'll make sure to keep that in mind when helping you plan trips in the future!

💾 Saving session 1 to Memory Bank...
✅ Session 1 saved.

--- Starting a brand new Session 2 for the SAME user ---

🗣️ You: Great. Now, can you plan a weekend dinner trip to Napa Valley?
🤖 Assistant: Great! I remember that you are a vegetarian, and I'll keep that in mind while planning your trip.

Here's a possible plan for a delightful weekend dinner trip to Napa Valley, focusing on excellent dining experiences with vegetarian options:

---

**Weekend Dinner Trip to Napa Valley (Vegetarian-Friendly)**

**Accommodation Suggestion:**
*   Consider staying at a bo

In [ ]:
# This time, we wrap the agent in AdkApp to enable tracing
app_with_tracing = reasoning_engines.AdkApp(agent=memory_agent, enable_tracing=True)
deployment_name = f"trip-planner-memory-{uuid4().hex[:6]}"

print(f"🚀 Deploying '{deployment_name}' with tracing enabled...")
deployed_agent = agent_engines.create(
    display_name=deployment_name,
    agent_engine=app_with_tracing,
    requirements=["google-cloud-aiplatform[adk,agent_engines]"],
    description="A trip planning agent with long-term memory and tracing enabled."
)
print("-" * 50)
print("✅ Agent with Memory and Tracing deployed successfully!")
display(Markdown(f"**Resource Name:** `{deployed_agent.resource_name}`"))

INFO:vertexai.agent_engines:Identified the following requirements: {'google-cloud-aiplatform': '1.105.0', 'pydantic': '2.11.7', 'cloudpickle': '3.1.1'}
INFO:vertexai.agent_engines:The following requirements are appended: {'pydantic==2.11.7', 'cloudpickle==3.1.1'}
INFO:vertexai.agent_engines:The final list of requirements: ['google-cloud-aiplatform[adk,agent_engines]', 'pydantic==2.11.7', 'cloudpickle==3.1.1']


🚀 Deploying 'trip-planner-memory-1c33a2' with tracing enabled...


INFO:vertexai.agent_engines:Using bucket neon-emitter-458622-e3-adk-staging
INFO:vertexai.agent_engines:Wrote to gs://neon-emitter-458622-e3-adk-staging/agent_engine/agent_engine.pkl
INFO:vertexai.agent_engines:Writing to gs://neon-emitter-458622-e3-adk-staging/agent_engine/requirements.txt
INFO:vertexai.agent_engines:Creating in-memory tarfile of extra_packages
INFO:vertexai.agent_engines:Writing to gs://neon-emitter-458622-e3-adk-staging/agent_engine/dependencies.tar.gz
INFO:vertexai.agent_engines:Creating AgentEngine
INFO:vertexai.agent_engines:Create AgentEngine backing LRO: projects/680476413759/locations/us-central1/reasoningEngines/5650706914489663488/operations/4616834447264710656
INFO:vertexai.agent_engines:View progress and logs at https://console.cloud.google.com/logs/query?project=neon-emitter-458622-e3
INFO:vertexai.agent_engines:AgentEngine created. Resource name: projects/680476413759/locations/us-central1/reasoningEngines/5650706914489663488
INFO:vertexai.agent_engines:

--------------------------------------------------
✅ Agent with Memory and Tracing deployed successfully!


**Resource Name:** `projects/680476413759/locations/us-central1/reasoningEngines/5650706914489663488`

---

## 5. Comparing Agents in Cloud Trace
Now, let's see the difference tracing makes.

1.  Navigate to the **[Cloud Trace](https://console.cloud.google.com/traces)** page in your Google Cloud Console.
2.  In the filter or search bar, look for traces related to your Reasoning Engines. You can often filter by the service name, which will contain your deployment name.

You will find detailed, step-by-step execution graphs for **`trip-planner-memory-...`** (Agent #2). However, you will find **no detailed traces** for **`trip-planner-basic-...`** (Agent #1), because we deployed it without enabling tracing. This demonstrates the critical importance of the `AdkApp` wrapper for observability and debugging.

### Query The Remote Agent
We'll have one conversation to teach the agent a preference, and then start a completely **new session** to see if it remembers.

In [ ]:
agent_client = agent_engines.get(deployed_agent.resource_name)
user_id_memory = f"memory-test-user-{uuid4().hex[:6]}"

# --- Conversation 1: Teach the Agent a Fact ---
print(f"--- 📚 Starting Conversation 1 for user '{user_id_memory}' (The 'Teach Me' Session) ---")
session1 = agent_client.create_session(user_id=user_id_memory)
query1 = "For all my future trips, please remember that I am a vegetarian."
print(f"💬 User: {query1}")
stream1 = agent_client.stream_query(message=query1, user_id=user_id_memory, session_id=session1['id'])
response1 = stream_and_print_response(stream1)

print("\n" + "-" * 50)

# --- Conversation 2: Test the Agent's Recall in a NEW Session ---
print(f"--- 🤔 Starting Conversation 2 for the SAME user (The 'Recall' Session) ---")
session2 = agent_client.create_session(user_id=user_id_memory) # A brand new session!
query2 = "Hi, I'm back. Can you suggest some dinner options for my trip to Napa Valley?"
print(f"💬 User: {query2}")
stream2 = agent_client.stream_query(message=query2, user_id=user_id_memory, session_id=session2['id'])
response2 = stream_and_print_response(stream2)

print("\n" + "-" * 50)
print("✅ Memory test complete. The agent should have NOT remember the vegetarian preference in its second response.")

--- 📚 Starting Conversation 1 for user 'memory-test-user-d9f856' (The 'Teach Me' Session) ---
💬 User: For all my future trips, please remember that I am a vegetarian.
🤖 Agent Response: Okay, I've noted that you are a vegetarian. I'll be sure to keep that in mind for all your future trip planning!

--------------------------------------------------
--- 🤔 Starting Conversation 2 for the SAME user (The 'Recall' Session) ---
💬 User: Hi, I'm back. Can you suggest some dinner options for my trip to Napa Valley?
🤖 Agent Response: Welcome back! I can certainly help with dinner options for your trip to Napa Valley.

I don't have any specific dining preferences saved for you yet. To help me give the best recommendations, could you tell me:

1.  What kind of cuisine are you interested in (e.g., American, Italian, French, farm-to-table)?
2.  What's your preferred price range (e.g., casual/affordable, mid-range, fine dining)?
3.  Are there any dietary restrictions or preferences I should be aware o

---

## 6. Comprehensive Evaluation of Agent #1

Now that we understand how to build and deploy agents, let's circle back to our first agent—the multi-agent trip planner. Because we gave it **custom tools with specific arguments**, we can now test it on a deeper level. We'll check if it:
1.  Chooses the correct sub-agent (`activities_agent` or `food_agent`).
2.  Calls the correct underlying tool (`get_travel_attractions` or `get_restaurant_recommendations`).
3.  Passes the **correct arguments** (`location`, `interest`, `cuisine`) to that tool.

### Helpers and Dataset for a Deeper Evaluation

In [ ]:
# --- Helper functions specifically for the evaluation process ---

def parse_multi_agent_stream_for_eval(stream_generator) -> dict:
    """Parses the stream, capturing full tool call details."""
    trajectory = []
    # This loop captures the tool calls made by the agent during its execution
    for chunk in stream_generator:
        try:
            # The tool call info is in the 'function_call' part of a chunk
            for part in chunk.get('content', {}).get('parts', []):
                if 'function_call' in part:
                    tool_call = part['function_call']
                    info = {
                        "tool_name": tool_call.get('name'),
                        "tool_input": dict(tool_call.get('args', {}))
                    }
                    if info not in trajectory: trajectory.append(info)
        except (KeyError, IndexError): pass
    return {"predicted_trajectory": trajectory}

def query_trip_planner_for_eval(prompt: str) -> dict:
    """Queries Agent #1 and formats the output for the evaluation service."""
    # We need to provide a user_id even for evaluation runs
    eval_user_id = f"eval-user-{uuid4().hex[:6]}"
    stream_generator = deployed_agent_no_trace.stream_query(message=prompt, user_id=eval_user_id)
    result = parse_multi_agent_stream_for_eval(stream_generator)
    result["predicted_trajectory"] = json.dumps(result.get("predicted_trajectory", []))
    return result

def display_eval_report(eval_result: pd.DataFrame):
    """Displays a formatted summary of the evaluation results."""
    display(Markdown("### Summary Metrics"))
    display(pd.DataFrame(eval_result.summary_metrics.items(), columns=["metric", "value"]))
    if hasattr(eval_result, "metrics_table"):
        display(Markdown("### Row-wise Metrics (Sample)"))
        display(eval_result.metrics_table.head())

def plot_bar_plot(eval_result: pd.DataFrame, title: str, metrics: list[str] = None):
    """Generates a bar plot for the given evaluation metrics."""
    fig = go.Figure()
    summary_metrics = eval_result.summary_metrics
    if metrics:
        summary_metrics = {k: v for k, v in summary_metrics.items() if any(m in k for m in metrics)}
    fig.add_trace(go.Bar(x=list(summary_metrics.keys()), y=list(summary_metrics.values()), name=title))
    fig.update_layout(barmode="group", title_text=title)
    fig.show()

# --- Create the detailed dataset for evaluation ---
eval_data = {
    "prompt": [
        "Find a historical site in Rome and a place for pasta.",
        "I'm in Tokyo and want to see a museum.",
        "I need a seafood restaurant in San Francisco.",
    ],
    # The ground truth now includes the tool and its exact arguments
    "reference_trajectory": [
        json.dumps([
            {"tool_name": "get_travel_attractions", "tool_input": {"location": "Rome", "interest": "historical site"}},
            {"tool_name": "get_restaurant_recommendations", "tool_input": {"location": "Rome", "cuisine": "pasta"}}
        ]),
        json.dumps([
            {"tool_name": "get_travel_attractions", "tool_input": {"location": "Tokyo", "interest": "museum"}}
        ]),
        json.dumps([
            {"tool_name": "get_restaurant_recommendations", "tool_input": {"location": "San Francisco", "cuisine": "seafood"}}
        ]),
    ],
}
eval_trip_planner_dataset = pd.DataFrame(eval_data)
print("✅ Detailed evaluation dataset created.")

✅ Detailed evaluation dataset created.


### Run the Evaluation
We'll use the `trajectory_exact_match` metric, which is very strict. It will only score a 1.0 if the agent calls the exact tools with the exact arguments in the correct order.

In [ ]:
# Define the metrics for our multi-agent system
tool_eval_metrics = [
    "trajectory_exact_match", # Checks for perfect match of tools, args, and order.
    "trajectory_recall",      # Checks what percentage of required calls were made.
]

tool_eval_task = EvalTask(
    dataset=eval_trip_planner_dataset,
    metrics=tool_eval_metrics,
    experiment=EVAL_EXPERIMENT_NAME
)

print("\n🚀 Running detailed tool evaluation... This may take several minutes.")

tool_eval_result = tool_eval_task.evaluate(
    runnable=query_trip_planner_for_eval,
    experiment_run_name=f"detailed-tool-eval-{uuid4().hex[:6]}"
)

print("\n✅ Evaluation complete!")
display_eval_report(tool_eval_result)
plot_bar_plot(
    tool_eval_result,
    title="Agent Tool-Calling Accuracy",
    metrics=["trajectory_exact_match/mean", "trajectory_recall/mean"],
)


🚀 Running detailed tool evaluation... This may take several minutes.


100%|██████████| 3/3 [00:27<00:00,  9.11s/it]
INFO:vertexai.preview.evaluation._evaluation:All 3 responses are successfully generated from the runnable.
INFO:vertexai.preview.evaluation._evaluation:Computing metrics with a total of 6 Vertex Gen AI Evaluation Service API requests.
100%|██████████| 6/6 [00:01<00:00,  5.01it/s]
INFO:vertexai.preview.evaluation._evaluation:All 6 metric requests are successfully computed.
INFO:vertexai.preview.evaluation._evaluation:Evaluation Took:1.208844171000237 seconds



✅ Evaluation complete!


### Summary Metrics

,metric,value
0,row_count,3.000000
1,trajectory_exact_match/mean,0.000000
2,trajectory_exact_match/std,0.000000
3,trajectory_recall/mean,1.000000
4,trajectory_recall/std,0.000000
5,latency_in_seconds/mean,14.880418
6,latency_in_seconds/std,10.772092
7,failure/mean,0.000000
8,failure/std,0.000000


### Row-wise Metrics (Sample)

,prompt,reference_trajectory,response,latency_in_seconds,failure,predicted_trajectory,trajectory_exact_match/score,trajectory_recall/score
0,Find a historical site in Rome and a place for...,"[{""tool_name"": ""get_travel_attractions"", ""tool...",,27.318752,0,"[{""tool_name"": ""transfer_to_agent"", ""tool_inpu...",0.0,1.0
1,I'm in Tokyo and want to see a museum.,"[{""tool_name"": ""get_travel_attractions"", ""tool...",,8.723366,0,"[{""tool_name"": ""transfer_to_agent"", ""tool_inpu...",0.0,1.0
2,I need a seafood restaurant in San Francisco.,"[{""tool_name"": ""get_restaurant_recommendations...",,8.599136,0,"[{""tool_name"": ""transfer_to_agent"", ""tool_inpu...",0.0,1.0


---

## 7. Cleanup

Finally, let's clean up the resources we've created to avoid incurring unnecessary costs. This will delete the deployed Agent Engines and the Vertex AI Experiment.

In [ ]:
# --- Delete the deployed Agent Engines ---
all_deployments = [deployed_agent_no_trace, deployed_agent]
for deployment in all_deployments:
    try:
        deployment.delete()
        print(f"🗑️ Deleted Agent Engine: {deployment.display_name}")
    except Exception as e:
        print(f"Could not delete {deployment.display_name}. It may have been deleted already. Error: {e}")